In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_unstained4"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\unstained_"

num_images=10000

image_dim=64 #Dim of the final images

nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_smallerfile2"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\smaller_file2_"

num_images=10000

image_dim=64 #Dim of the final images

nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_sample1"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\sample1_"

image_dim=64 #Dim of the final images

nuclear_channel=None
cellmask_channel="Ch1_mask"

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\Retina_1_2"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\retina_1_2_"

image_dim=64 #Dim of the final images

nuclear_channel=None
cellmask_channel="Ch1_mask"
cell_info_str="\\retina_1_2.csv"

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\Colon"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\colon_"

if not os.path.exists(outpath):
  os.makedirs(outpath)

image_dim=64 #Dim of the final images

nuclear_channel=None
cellmask_channel="Ch1_mask"
cell_info_str="\\colon.csv"

In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\Retina_0_0"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\retina_0_0_"

if not os.path.exists(outpath):
  os.makedirs(outpath)

image_dim=128 #Dim of the final images

nuclear_channel=None
cellmask_channel="Ch1_mask"
cell_info_str="\\Retina_0_0.csv"

In [ ]:
#Read in images
names=[]
for entry in os.listdir(readpath): #Read all photos
    if os.path.isfile(os.path.join(readpath, entry)):
        names.append(entry)
       # if len(names)>=num_images*3:
        #    break

channel_names=[channel_name[:-8] for channel_name in np.unique([name.split("_")[file_prefix.count("_")+1] for name in names])] 

print("Found",len(channel_names),"channel(s)")
print(channel_names)

data_dict={}
for name in names:
    cell_num=name.split("_")[file_prefix.count("_")]
    if cell_num not in data_dict.keys():
        data_dict[cell_num]=[cv.imread(readpath+file_prefix+cell_num+'_'+channel_name+'.ome.tif',0  | cv.IMREAD_ANYDEPTH) for channel_name in channel_names]

#13s

In [ ]:
len(data_dict.keys())

In [ ]:
BUGS=[]
for image in data_dict:
    for i in data_dict[image]:
      #  print(i)
        try: 
            if i==None:
                BUGS.append(image)
        except:
            pass

print(len(BUGS))

In [ ]:
[data_dict.pop(BUG) for BUG in BUGS]

In [ ]:
#Rescale OME images to tiff
max_mins=[[0,1e6] for _ in channel_names]
for image_i in data_dict:
    image=data_dict[image_i]
    for i,_ in enumerate(max_mins):
        if max_mins[i][0]<np.max(image[i]):
            max_mins[i][0]=np.max(image[i])
        if max_mins[i][1]>np.min(image[i]):
            max_mins[i][1]=np.min(image[i])
print("Maximum and minimums of every raw channel")
print(max_mins)

In [ ]:
#del data_dict[image_i] Use in case of nonetype error

In [ ]:
def scale(img,max_min):
    return ((img-max_min[1])/(max_min[0]-max_min[1])*255)#.astype(np.uint8)

image_dict={} 
maskpath=basepath+"\\ExportedMasks"
for image_i in data_dict:
    images=data_dict[image_i]  

    processed_images=[]
    for i,image in enumerate(images):
        processed_images.append(scale(image,max_mins[i]))  #Rescale the images

    if cellmask_channel is not None:
        for channel in channel_names:
            processed_images.append(cv.imread(maskpath+file_prefix+image_i+'_'+channel+'.dmask.pgm',0 ))  # mask for channel

    image_dict[image_i]=processed_images

print("Order of the channels:")
channel_order=channel_names+[channel+'_mask' for channel in channel_names]
print(channel_order)


#This took 3 mins for 10,000 images and 7 channels

In [ ]:
#Create dataframe of image shape information
shape_df = pd.DataFrame()
X=[]
Y=[]
names=[]
for image_i in image_dict:
    x,y = image_dict[image_i][0].shape
    X.append(x)
    Y.append(y)
    names.append(image_i)
shape_df['image']=names
shape_df['x']=X
shape_df['y']=Y

In [ ]:
#Investigate sizes of images
print('Average width of image: ', np.round(shape_df.x.mean(), 0), ' Min: ', shape_df.x.min(), ' Max: ', shape_df.x.max())
print('Average height of image: ', np.round(shape_df.y.mean(), 0), ' Min: ', shape_df.y.min(), ' Max: ', shape_df.y.max())
fig, axs = plt.subplots(2, sharex=True, figsize=(10, 5))
fig.suptitle('Raw image shapes')
numbins = 30
axs[0].hist(shape_df.x, bins=numbins);
axs[1].hist(shape_df.y, bins=numbins);
plt.savefig(basepath+"\\dimension_hist.png")

In [ ]:
df=pd.DataFrame()
df['Cell_ID']=image_dict.keys()

In [ ]:
len(df)

In [ ]:
def Touches_Boundary(data_dict,channel_order,cellmask_channel,threshold=2):
    i=channel_order.index(cellmask_channel)
    touches=[]
    touch_val=[]
    for image_i in data_dict:
        image=data_dict[image_i][i].copy()
        image[1:-1,1:-1]=0  
        edge_pos=np.count_nonzero(image) 
        touch_val.append(edge_pos)
        if touch_val[-1]>threshold:
            touches.append(1)
        else:
            touches.append(0)

    plt.hist(np.log(np.array(touch_val)+1),bins=100);
    plt.axvline(x=threshold,color="red")
    plt.show()

    return touches


df["Touches_boundary"]=Touches_Boundary(image_dict,channel_order,cellmask_channel,threshold=2)


In [ ]:
#This sets all the backgrounds to 0
cropped_dict={}
for channel in channel_names:
    channel_index=channel_order.index(channel)
    cellmask_index=channel_order.index(channel+'_mask')

    for image in image_dict:
        mean=np.mean(image_dict[image][channel_index][image_dict[image][cellmask_index]==0])
        image_dict[image][channel_index]-=(mean)
        a,b,c,d=cv.boundingRect(image_dict[image][cellmask_index])
        cropped_img=image_dict[image][channel_index][b:d+b,a:c+a]
        cropped_dict[image]=cropped_img

final_images=image_dict.copy()

In [ ]:
# plt.imshow(image_dict['1'][0])
# plt.show()
# plt.imshow(cropped_dict['1'])

# #Create dataframe of image shape information
# shape_df = pd.DataFrame()
# X=[]
# Y=[]
# names=[]
# for image_i in cropped_dict:
#     x,y = cropped_dict[image_i].shape
#     X.append(x)
#     Y.append(y)
#     names.append(image_i)
# shape_df['image']=names
# shape_df['x']=X
# shape_df['y']=Y

# #Investigate sizes of images
# print('Average width of image: ', np.round(shape_df.x.mean(), 0), ' Min: ', shape_df.x.min(), ' Max: ', shape_df.x.max())
# print('Average height of image: ', np.round(shape_df.y.mean(), 0), ' Min: ', shape_df.y.min(), ' Max: ', shape_df.y.max())
# fig, axs = plt.subplots(2, sharex=True, figsize=(10, 5))
# fig.suptitle('Raw image shapes')
# numbins = 30
# axs[0].hist(shape_df.x, bins=numbins);
# axs[1].hist(shape_df.y, bins=numbins);


In [ ]:
len(image_dict)

In [ ]:
# for image_i in cropped_dict:
#     image = cropped_dict[image_i]
#     x,y = np.shape(image)
#     s = max(x, y)
#     width = int((s-x)/2)
#     height = int((s-y)/2)
#     image = cv.copyMakeBorder(image, width, width, height, height, cv.BORDER_CONSTANT)
#     res1 = cv.resize(image, dsize=(image_dim, image_dim), interpolation=cv.INTER_CUBIC)
#     cropped_dict[image_i] = [res1]

In [ ]:
#Pad images to squares
maxi=shape_df.x.max()
mini=shape_df.x.min()

def g(x):
    return 1/(np.log(x))**10

def f(x):
    return (g(x)-g(maxi))*(image_dim-mini)/(g(mini)-g(maxi))

def r(x):
    return int(round(x,0))

In [ ]:
X=np.linspace(mini,maxi,num=1000)
plt.plot(X,f(X))
#plt.plot(X,X+f(X),color="red")
plt.xlabel("Longest dimension of the image (pixels)")
plt.ylabel("Padding Width (pixels)");
plt.savefig(basepath+"\\dimension_padding.png")

In [ ]:
for image_i in final_images:
    images = final_images[image_i]
    x,y = np.shape(final_images[image_i][0])
    if y>x:
        height=f(y)
        width=(y+height-x)
    else:
        width=f(x)
        height=(x+width-y)

    for i,image in enumerate(images):
        padded_img=cv.copyMakeBorder(image, r(width/2), r(width/2), r(height/2), r(height/2), cv.BORDER_CONSTANT)
        resized_img=cv.resize(padded_img,(image_dim,image_dim))
        final_images[image_i][i]=resized_img

#14 min 

In [ ]:
c_dict={}
for image_i in cropped_dict:
    
    image = cropped_dict[image_i]
    x,y = np.shape(image)
    if y>x:
        height=f(y)
        width=(y+height-x)
    else:
        width=f(x)
        height=(x+width-y)

    padded_img=cv.copyMakeBorder(image, r(width/2), r(width/2), r(height/2), r(height/2), cv.BORDER_CONSTANT)
    resized_img=cv.resize(padded_img,(image_dim,image_dim))
    c_dict[image_i]=resized_img

In [ ]:
del cropped_dict[image_i]

In [ ]:
len(c_dict)

In [ ]:
df5=pd.read_csv(basepath+cell_info_str)

In [ ]:
def DNA_POS(data_dict,channel_order,nuclear_channel,threshold=0.8):
    if nuclear_channel is None:
        df5=pd.read_csv(basepath+cell_info_str)
        df5.rename(columns = {'Object Number':'Cell_ID'}, inplace = True)
        df["Cell_ID"]=pd.to_numeric(df['Cell_ID'])
        int_df=pd.merge(df,df5,how='inner',on=["Cell_ID"]) 
        nuc_intensity=int_df["Intensity_MC_Ch07"].to_numpy()  #CHANGE THIS TO NUCLEAR CHANNEL
        nuc_intensity_s=np.sort(nuc_intensity)
        plt.hist(nuc_intensity_s[:-5000],bins=100);
        plt.axvline(x=threshold,color="red")
        plt.show()
        dna_pos=nuc_intensity>=threshold
        int_df["DNA_pos"]=dna_pos

        return int_df

    else:

        i=channel_order.index(nuclear_channel)
        dna_pos=[]
        mean_nuc=[]
        for image_i in data_dict:
            image=data_dict[image_i][i]
            mean_nuc.append(np.mean(image))
            if mean_nuc[-1]>=threshold:
                dna_pos.append(1)
            else:
                dna_pos.append(0)

        plt.hist(mean_nuc,bins=100);
        plt.axvline(x=threshold,color="red")
        plt.show()
            
    return dna_pos

#df["DNA_pos"]=DNA_POS(final_images,channel_order,nuclear_channel,threshold=0.8)
df6=DNA_POS(final_images,channel_order,nuclear_channel,threshold=100000)


In [ ]:
len(df6)

In [ ]:
posx,posy=[],[]
n=100 
num_images=(len(final_images)//n)*n
def make_tile(final_images,channel_order,prefix=""):
    for coi,coi_str in enumerate(channel_order):  #coi is channel of interest
        print("Saving "+coi_str+"...")
        display_ims=[]
        names=[]
        for image_i in final_images:
            images = final_images[image_i]
            display_ims.append(images[coi])
            names.append(image_i)
        #x axis of final image or array 
        posx,posy=[],[]
        l=num_images//n
        for i in range(l):
            for j in range(n):
                posx.append(i)
                posy.append(j)
                if j==0:
                    col_array=display_ims[i*n]
                else:
                    col_array=np.hstack((col_array,display_ims[i*n+j]))
            if i==0:
                row_array=col_array
            else:
                row_array=np.vstack((row_array,col_array))
        # myfig=plt.figure(dpi=2000)   #Only for small number of images  
        # plt.imshow(row_array-np.min(row_array),vmin=0,vmax=70,cmap="Greys")
        # plt.axis('off')
        # plt.savefig(outpath+"\\"+coi_str+".png",bbox_inches="tight")
        # plt.close()
        np.save(outpath+"\\"+prefix+coi_str,row_array)
    np.save(outpath+"\\"+prefix+"image_ID",names)
    #This took 5min 16s for 10,000 images and 7 channels (+7 masks)

    return posx,posy



In [ ]:

posx,posy=make_tile(final_images,channel_order)

df6=df6.iloc[:num_images]
df6["POSX"]=posx[:num_images]
df6["POSY"]=posy[:num_images]
df6["X_length"]=shape_df['x'][:num_images]
df6["Y_length"]=shape_df['y'][:num_images]
df6.to_csv(outpath+"\\"+""+"cell_info.csv")

In [ ]:
#make_tile(final_images,['Ch1_mask'])
#make_tile(final_images,channel_order)
#make_tile(cropped_dict,['Ch1'],prefix="cropped_")